In [12]:
import pandas as pd
import numpy as np
import math
import datetime
import ipynb.fs.defs.Local_search

#table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
#time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
#dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
#fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
#fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1")
#fur1d = pd.read_csv("../Dati/turni_f/furgoncino1_KM_routing.csv", delimiter=";", encoding = "ISO-8859-1")
#fur2d = pd.read_csv("../Dati/turni_f/furgoncino2_KM_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur1 = pd.read_csv("../Dati/turni_f/2-opt1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur2 = pd.read_csv("../Dati/turni_f/2-opt2_shifts.csv", delimiter=";", encoding = "ISO-8859-1")

In [13]:
week_fur1 = ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
week_fur2 = ['martedi', 'mercoledi', 'giovedi']
for day_1 in week_fur1:
    fur1_copy = fur1[day_1][:]
    length = len(fur1_copy.dropna())
    i = 0
    to_insert_list = fur1[day_1][:]
    while i < length:
        time = sum(new_route(fur1_copy, dft).values)
        to_insert = to_insert_list.pop(0)
        rollback_index = fur1_copy.index(to_insert)
        fur1_copy.remove(to_insert)
        removed_city_time = sum(new_route(fur1_copy, dft).values)
        #Improvement with remove of to_insert (original time - removed city time)
        day_diff = time-removed_city_time
        best_day_time = -300000
        #Find Best position in fur1 (among every day) and get time difference before and after insert.
        for day_fur1 in week_fur1:
            if day_1 != day_fur1 and !fur1[day_fur1].contains(to_insert):
                #Get worsening before and after insert of to_insert
                fur1_day_fur1_time = sum(new_route(fur1[day_fur1], dft).values)
                fur1_day_fur1_copy = fur1[day_fur1][:]
                idx = best_index(fur1[day_fur1], new_route(fur1[day_fur1], dft), to_insert, dft)
                fur1_day_fur1_copy.insert(idx, to_insert)
                fur1_day_fur1_copy_time = sum(new_route(fur1_day_fur1_copy, dft).values)
                #Best_day_time contains best improvement-worsening results
                if day_diff + (fur1_day_fur1_time-fur1_day_fur1_copy_time) > best_day_time:
                    best_day = day_fur1
                    best_idx = idx
                    best_day_time = day_diff + (fur1_day_fur1_time-fur1_day_fur1_copy_time)
                    fur = 1
                    
        #Find Best position in fur2 (among every day) and get time difference before and after insert.            
        for day_fur2 in week_fur2:           
            if day_1 != day_fur2 and !fur2[day_fur2].contains(to_insert):
                #Get worsening before and after insert of to_insert
                fur2_day_fur2_time = sum(new_route(fur2[day_fur2], dft).values)
                fur2_day_fur2_copy = fur2[day_fur2][:]
                idx = best_index(fur2[day_fur2], new_route(fur2[day_fur2], dft), to_insert, dft)
                fur2_day_fur2_copy.insert(idx, to_insert)
                fur2_day_fur2_copy_time = sum(new_route(fur2_day_fur2_copy, dft).values)
                #Best_day_time contains best improvement-worsening results
                if day_diff + (fur2_day_fur2_time-fur2_day_fur2_copy_time) > best_day_time:
                    best_day = day_fur2
                    best_idx = idx
                    best_day_time = day_diff + (fur2_day_fur2_time-fur2_day_fur2_copy_time)
                    fur = 2
        #If gain improvement insert new location in best location of relative day and pickup
        if best_day_time > 0:
            fur1[day_1] = fur1_copy[:]
            if(fur == 1):
                fur1[best_day].insert(best_idx, to_insert)
            elif(fur == 2)
                fur2[best_day].insert(best_idx, to_insert)
        #If there isn't an improvement rollback original schedule
        else:
            fur1_copy.insert(rollback_index, to_insert)
        i += 1